In [73]:
import random
import numpy as np
import timeit
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime
import pandas as pd
import time

In [7]:
random.randint(0,10) # random int between 1st and 2nd param, inclusive
random.randrange(-0,10,3) # number by "step" specified in 3rd param, between 1st and 2nd params
random.uniform(-10,10) # not a whole integer

1